In [74]:
from LanguageModels.BERT import BERT
from LanguageModels.Word2Vec import Word2Vec
from Preprocessing.DataLoader import DataLoader
from Preprocessing.LemmatizerPreprocessor import LemmatizerPreprocessor
import torch
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np


In [75]:
is_cuda = True
is_cuda = is_cuda and torch.cuda.is_available() # will turn off cuda if the machine doesnt have a GPU

In [76]:
lp = LemmatizerPreprocessor()

In [77]:
# Load data
data = DataLoader('../data/EMNLP2020.csv').load()

In [78]:
## Bert
bert = BERT(cuda = is_cuda)

## W2V
w2v = Word2Vec(path='../data/glove.6B/glove.6B.200d.txt') 

In [79]:
#### multiclass case
X_multiclass_w2v, y_multiclass_w2v = w2v.featurize(data, lp, mode='multiclass', remove_neg_samples=True)

In [80]:
X_multiclass_bert, y_multiclass_bert = bert.featurize(data, lp, mode='multilabel', remove_neg_samples=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1450/1450 [01:31<00:00, 15.92it/s]


In [81]:
X, y = X_multiclass_bert, y_multiclass_bert

In [82]:
print(X.shape)
print(y.shape)

(750, 768)
(750, 2)


In [133]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [134]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(502, 768)
(248, 768)
(502, 2)
(248, 2)


In [135]:
def ymax_2arg(y):
    a, b = y
    
    if a == None:
        return 0
    elif b == None:
        return 1
    else:
        return np.argmax(y)

In [136]:
y_train_classes = []
for i in range(len(y_train)):
    y_train_classes.append(ymax_2arg(y_train[i]))

y_test_classes = []
for i in range(len(y_test)):
    y_test_classes.append(ymax_2arg(y_test[i]))


In [147]:
clf = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=0.01, gamma='scale'))
clf.fit(X_train, y_train_classes)
print("Training Accuracy: ", accuracy_score(y_train_classes, clf.predict(X_train)))
print("Testing Accuracy: ", accuracy_score(y_test_classes, clf.predict(X_test)))
print(classification_report(y_test_classes, clf.predict(X_test)))

Training Accuracy:  0.8844621513944223
Testing Accuracy:  0.8790322580645161
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        30
           1       0.88      1.00      0.94       218

    accuracy                           0.88       248
   macro avg       0.44      0.50      0.47       248
weighted avg       0.77      0.88      0.82       248



C:\Users\Ryan\anaconda3\envs\python3_env\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ryan\anaconda3\envs\python3_env\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ryan\anaconda3\envs\python3_env\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
